# Продвинутый Python, лекция 11

**Лектор:** Петров Тимур

**Семинаристы:** Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся, Садуллаев Музаффар

**NB:** код внутри колаба не запустится

## API

Итак, давайте вначале быстренько напомним, что такое API, и дальше к телеге перейдем

Что такое API?

API (Application Programming Interface) - по существу это контракт, который предоставляется пользователю того или иного сервиса. Что это обозначает?

* Сервис говорит: ты вызываешь вот такую ручку (функцию), передавая такие данные, я тебе выдаю ответ

* Мы говорим: мы даем вот так вот данные, забирай, ожидаем от тебя ответа

По сути - набор функций, который предоставляет сервис. Эти функции мы вызываем либо напрямую, либо скрытно через использование других функций

Что такое в таком случае какой-нибудь REST API? Это API, которое соблюдает REST-стиль, в целом все

А теперь можно и про телегу поговорить

## Telegram API

Телеграм, как и множество других сервисов, имеет собственное API: [тык](https://core.telegram.org/) - это вот тот самый набор ручек, через которые мы можем выполнять различные действия. Как это применимо к нашему питончику?

А очень просто - python-бибилотеки есть обертки над взаимодействием с данным API, созданный достаточно часто как 1 к 1 с исходным API.

Вариантов библиотек много, вот часть из них:

* https://docs.python-telegram-bot.org/en/stable/index.html - PyTelegramBot

* https://aiogram.dev/ - AioGram

* https://docs.pyrogram.org/ - PyroGram


Чем они отличаются? Если вкапываться в суть, то не особо они и отличаются: общий предок - это Telegram API, дальше в зависимости от того, как и что обрабатывается внутри, мы получаем немного различный интерфейс и способ вазимодействия. Что может быть важно: не все поддерживают асинхронность (до определенной версии тот же PyTelegramBot был неасинхронный, сейчас вроде как он уже работает и так)

Вариаций много и разных, в нашем случае мы будем пользоваться aiogram.


## Aiogram

Ссылка на документацию: [Тык](https://docs.aiogram.dev/en/stable/)



In [ ]:
!pip install aiogram

Сегодня попробуем создать не очень полезного бота, в которого напихнем кучу всего разного-интересного. Поехали!

### Шаг 1. Создать бота в Телеге

Чтобы управлять ботом, нужно его создать. Делаем это в телеграме через @BotFather - отдельный бот для создания ботов

Через /newbot придумываем имя и название (которое будет отображаться через @), далее получаем token - идентификатор, с помощью которого мы сможем управлять командами внутри нашего бота.


Создаем самого простого бота (умеет только говорить Hello + ваше имя):


In [ ]:
import asyncio
import logging
import sys
from os import getenv

from aiogram import Bot, Dispatcher, html
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.filters import CommandStart
from aiogram.types import Message

TOKEN = '7826404927:AAHkbNPz1Jy8L-_MyRatl3XeOtS5zqOMRWA'

dp = Dispatcher()

@dp.message(CommandStart())
async def command_start_handler(message: Message) -> None:
    """
    This handler receives messages with `/start` command
    """
    await message.answer(f"Hello, {html.bold(message.from_user.full_name)}!")

async def main() -> None:
    # Initialize Bot instance with default bot properties which will be passed to all API calls
    bot = Bot(token=TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
    # And the run events dispatching
    await dp.start_polling(bot)

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, stream=sys.stdout)
    asyncio.run(main())

Что мы здесь сделали?

* Ввели наш токен (понятное дело, что вот так его хранить не надо, но для учебных целей подойдет)

* Создали бота (отдельный объект Bot)

* Создали диспетчера, который забирает входящий поток и отправляет его на обработку. По сути дела, мы и будем наделять этот диспетчер всей начинкой, которая происходит внутри бота

* Добавили хендлер, который отсылает сообщение (dp.message), внутри которого указываем, какие сообщения он должен обрабывать (в нашем случае это CommandStart - /start)

* Сделали start_polling - опросник (диспетчер ходит и слушает запросы к указанному боту)

Из полезного - у нас сразу же есть встроенный логгер, с помощью которого можно сразу понимать, что происходит

Сразу же понятно, зачем нам тут асинхронщина - при большей нагрузке это имеет смысл.

Ура, первый бот готов! Что дальше?

### Шаг 2. Учимся понимать, что происходит внутри

Итак, в handler у нас передается такая штука, как Message. Давайте разбираться, что там внутри находится:

https://docs.aiogram.dev/en/stable/api/types/message.html

На самом деле, передается дохрена всего. Давайте попробуем сделать несколько простых вещей:

* Вывести id пользователя (в качестве ответа пользователю)

* Сделать ответ на фотку, что она классная!

In [ ]:
import asyncio
import logging
import sys
from os import getenv

from aiogram import Bot, Dispatcher, html, F
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.filters import CommandStart
from aiogram.types import Message

TOKEN = '7826404927:AAHkbNPz1Jy8L-_MyRatl3XeOtS5zqOMRWA'

dp = Dispatcher()

@dp.message(CommandStart())
async def command_start_handler(message: Message) -> None:
    """
    This handler receives messages with `/start` command
    """
    await message.answer(f"Hello, {html.bold(message.from_user.full_name)}!")

@dp.message(F.text == 'my id')
async def command_my_id_handler(message: Message) -> None:
    await message.reply(f"Your id is: {message.from_user.id}!")

@dp.message(F.photo != None)
async def command_my_photo_handler(message: Message) -> None:
    await message.reply(f"Nice photo!")

async def main() -> None:
    # Initialize Bot instance with default bot properties which will be passed to all API calls
    bot = Bot(token=TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
    # And the run events dispatching
    await dp.start_polling(bot)

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, stream=sys.stdout)
    asyncio.run(main())

Как тут работать с фильтрами и в каком приоритете они будут работать?

Внутри aiogram есть отдельный модуль F, который и отвечает за фильтрацию. По сути, он принимает объект, который мы получаем и проверяет его по необходимомум условию. Далее, если результат True, то запускается обработка.

Что делать, если несколько handlerов получит нужный результат? Давайте пробовать:

In [ ]:
import asyncio
import logging
import sys
from os import getenv

from aiogram import Bot, Dispatcher, html, F
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.filters import CommandStart
from aiogram.types import Message

TOKEN = '7826404927:AAHkbNPz1Jy8L-_MyRatl3XeOtS5zqOMRWA'

dp = Dispatcher()

@dp.message(CommandStart())
async def command_start_handler(message: Message) -> None:
    """
    This handler receives messages with `/start` command
    """
    await message.answer(f"Hello, {html.bold(message.from_user.full_name)}!")

@dp.message(F.text == 'my id')
async def command_my_id_handler(message: Message) -> None:
    await message.reply(f"Your id is: {message.from_user.id}!")

@dp.message(F.photo != None)
async def command_my_photo_handler(message: Message) -> None:
    await message.reply(f"Nice photo!")

@dp.message(F.photo != None)
async def command_my_photo_handler_1(message: Message) -> None:
    await message.reply(f"Nice photo!!!")

async def main() -> None:
    # Initialize Bot instance with default bot properties which will be passed to all API calls
    bot = Bot(token=TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
    # And the run events dispatching
    await dp.start_polling(bot)
3
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, stream=sys.stdout)
    asyncio.run(main())

Ответ - линейная логика (что раньше, то и возьмет в обработку)

Так, ну хорошо, предлагаешь писать все в основном теле и страдать?

Конечно же нет! Давайте наводить порядок и в этом нам помогут роутеры. В чем разница?

Диспетчер (Dispatcher) - это центральный маршрутизтор для всех запросов. Поступает запрос - он его переводит в обработку куда надо

Роутер (Router) - это подсчасть маршрутизатора, который владеет своей частью внутри модуля

Соответственно логикак такая:

* Создаем на каждую часть свой собственный роутер

* Роутер добавляем к диспетчеру

* Profit!


![](https://docs.aiogram.dev/en/dev-3.x/_images/update_propagation_flow.png)

Итак, разделили отдельно наши хендлеры (один работает по сообщениям, другой по картинкам). В целом структура более-менее понятная

Что еще умеем в рамках наших ботов? Давайте добавим еще кастомную команду, с помощью которой будем выводить рандомного попугая, если в сообщении есть слово parrot (для чего этот курс еще нужен) или с помощью отдельной команды:

In [ ]:
parrots = {1: 'https://glavnoehvost.by/upload/medialibrary/b39/eco9kya7tmtlxkdbgky3cuyb3rbs0mfv.jpg',
           2: 'https://avatars.mds.yandex.net/get-mpic/11396862/2a0000018c290b5f7aa193f4ebf41c9c4b77/orig',
           3: 'https://kotsobaka.com/wp-content/uploads/2018/08/2748131046_8a253489b5_b.jpg',
           4: 'https://get.wallhere.com/photo/parrot-bird-color-feathers-1091582.jpg',
           5: 'https://img2.akspic.ru/attachments/originals/7/8/6/4/84687-ara-nerazluchnik-dlinnohvostyj_popugaj-roza_kolchatoj_popugaj-klyuv-2560x1600.jpg',
           6: 'https://avatars.mds.yandex.net/i?id=3bba204af3c534feb91529b1ff1b56f7_l-4809943-images-thumbs&n=13'}

@text_router.message(F.text.contains("parrot"))
async def text_my_parrot_handler(message: Message) -> None:
    await message.react(reaction=[{"type": "emoji", "emoji": "👀"}])
    await message.reply_photo(parrots[randint(1, 7)])

@text_router.message(Command("parrot"))
async def command_parrot_handler(message: Message) -> None:
    await message.reply_photo(parrots[randint(1, 7)])

Ура, успех! Что еще можно отправлять:

* Видосики

* Эмодзи

* Игры

* Контакты

И так далее, как вашей духе захочется. Что еще важно? Форматирование

Внутри себя aiogram проддерживает HTML-разметку, подробнее можно вот тут: https://docs.aiogram.dev/en/latest/utils/formatting.html

### Шаг 2.5 Параметры внутри команды

Мы не всегда хотим отправлять просто команды, в какие-то моменты нам нужно отправлять команду с параметрами (например, пришли мне сообщение через час).

Что нам с этим сделать? На самом деле, внутри уже есть обработка параметров, давайте на примере:

In [ ]:
from aiogram import Bot, Dispatcher, html, F, Router
from aiogram.filters import CommandStart, Command, CommandObject
from aiogram.types import Message

sch_router = Router()

@sch_router.message(Command(commands=["bomb"]))
async def add_bomb_timer(message: Message, command: CommandObject): ## Внутри любой команды есть CommandObject, в который передаются аргументы
    commands = command.args
    if commands is None:
        await message.answer(r"Вы не передали параметр. Попробуйте вот так: /bomb 10")
        return
    try:
        delay_t = int(commands.split()[0])
    except ValueError:
        await message.answer(r"Вы не передали правильный параметр. Попробуйте вот так: /bomb 10")
        return
    await message.answer(f"The clock is ticking! Timer: {delay_t} seconds")

### Шаг 2.9 Выбрасываем запросы

Если мы выключаем бота и продолжаем посылать сообщения, то при включении бота они все отработают (почему? Потому что запрос есть, они хранятся в очереди)

Чтобы такого сделать, дабы не перезагрузить все добро? 1 строчка:

In [ ]:
await bot.delete_webhook(drop_pending_updates=True) ## удаляем вебхук, удаляя все, что ожидает

### Шаг 3. Кнопки

Так, ну хорошо, было бы славно добавить обработку не только сообщений, но и добавить какой-нибудь красивый UI.

Есть два способа:

* Кнопки - то, что появляется внизу

* Инлайн-кнопки - кнопки, которые привязаны к сообщению (про них на семинаре)

Сейчас сделаем простые кнопки. У кнопочек самый простой дизайн: то, что тыкаем, мы отправляем в бота отдельным сообщением (которое дальше мы можем обрабатывать)

Давайте модифицируем наш start, чтоб он отправлял кнопочки:

In [ ]:
from aiogram import Bot, Dispatcher, html, F, Router
from aiogram.filters import CommandStart, Command
from aiogram.types import Message, KeyboardButton, ReplyKeyboardMarkup

@text_router.message(Command("start"))
async def cmd_start(message: Message):
    kb = [
        [KeyboardButton(text="parrot!!!")],
        [KeyboardButton(text="Get my id")]
    ]
    keyboard = ReplyKeyboardMarkup(keyboard=kb)
    await message.answer("Чего желаете?", reply_markup=keyboard)

Чет некрасиво выглядит (по дефолту он закидывает их поштучно сверху вниз как есть)

И давайте еще попробуем поменять текст (вместо "Write a message" сделаем что-нибудь другое):

In [ ]:
@text_router.message(Command("start"))
async def cmd_start(message: Message):
    kb = [
        [KeyboardButton(text="parrot!!!")],
        [KeyboardButton(text="Get my id")]
    ]
    keyboard = ReplyKeyboardMarkup(keyboard=kb, resize_keyboard=True, input_field_placeholder="Чего хочешь")
    ## resize_keayboard - передвинь по клавиатуре, input_field_placeholder - замена input
    await message.answer("Чего желаете?", reply_markup=keyboard)

@text_router.message(F.text == 'Get my id')
async def command_my_id_handler(message: Message) -> None:
    await message.reply(f"Your id is: {message.from_user.id}!")

Все равно не так красиво (две кнопки можно было бы сделать слева направо, а не сверху вниз). Ну, давайте чинить с помощью конструктора кнопок - ReplyKeyboardBuilder:

In [ ]:
@text_router.message(Command("start"))
async def cmd_start(message: Message):
    builder = ReplyKeyboardBuilder() ## создаем строение
    builder.add(KeyboardButton(text="parrot!!!")) ## добавляем кнопочки
    builder.add(KeyboardButton(text="Get my id"))
    builder.adjust(2) ## сколько кнопок на 1 строку (в целом можно сразу задать размер)
    await message.answer("Чего желаете?", reply_markup=builder.as_markup(resize_keyboard=True, input_field_placeholder="Чего хочешь")) ##переводим

Но вот таким образом можно задать только ровные кнопочки... А допустим, мы хотим что-то более веселое. Давайте это пробовать на примере еще специальных кнопок:

* Запрос геолокации

* Запрос контакта

* Создать опрос

In [ ]:
@text_router.message(Command("special_buttons"))
async def cmd_special_buttons(message: Message):
    builder = ReplyKeyboardBuilder()
    builder.row(
        KeyboardButton(text="Запросить геолокацию", request_location=True),
        KeyboardButton(text="Запросить контакт", request_contact=True)
    )
    builder.row(KeyboardButton(
        text="Создать викторину",
        request_poll=KeyboardButtonPollType(type="quiz"))
    )

    await message.answer(
        "Выберите действие:",
        reply_markup=builder.as_markup(resize_keyboard=True, input_field_placeholder="Чего хочешь"),
    )

## Шаг 4. Фильтры

Итак, давайте рассуждать, что нам еще нужно. Допустим, мы хотим делать мультифункционального бота: чтобы и в чате отвечал, и сам по себе что-то да делал, но при этом функционал будет разным, верно ведь?

Давайте сделаем вот такой функционал:

* Хотим уметь запускать рулетку в группах, но не в личном чате

Начнем с рулетки:

In [ ]:
from aiogram.enums.dice_emoji import DiceEmoji

@router.message(Command(commands=["dice"]))
async def cmd_dice(message: Message):
    await message.answer_dice(emoji=DiceEmoji.SLOT_MACHINE)

Опа, работает! Но в чате работает, а мы допустим такого не хотим

Что будем делать? А давайте добавлять фильтры!

In [ ]:
from typing import Union
from aiogram.filters import BaseFilter
from aiogram.types import Message


class ChatTypeFilter(BaseFilter): # Создаем базовый фильтр
    def __init__(self, chat_type: Union[str, list]):
        self.chat_type = chat_type #Какие нам нужны типы чатов

    async def __call__(self, message: Message) -> bool:
        if isinstance(self.chat_type, str):
            return message.chat.type == self.chat_type
        else:
            return message.chat.type in self.chat_type

In [ ]:
@text_router.message(ChatTypeFilter(chat_type=["group", "supergroup"]), Command(commands=["dice"]))
async def cmd_dice(message: Message):
    await message.answer_dice(emoji=DiceEmoji.SLOT_MACHINE)

Опа, работает в чате, но не работает в личке! Ух тыыыы

Но теперь структурно выглядит не очень: вот тут все вместе. Давайте заведем отдельно все только для групп (на роутер можно нацепить фильтр!)

In [ ]:
group_router = Router()
group_router.message.filter(
    ChatTypeFilter(chat_type=["group", "supergroup"])
)

@text_router.message(Command(commands=["dice"]))
async def cmd_dice(message: Message):
    await message.answer_dice(emoji=DiceEmoji.SLOT_MACHINE)

### Шаг 4. Middleware

![](https://docs.aiogram.dev/en/dev-3.x/_images/basics_middleware.png)

Что такое Middleware и нафиг он нужен? По сути, это прослойка между запросом и итоговым результатом (с помощью которого можно каким-то образом обогащать данные, фильтровать и так далее)

Аналог: Декораторы! (это же обертка по сути)

Middleware бывает 2 типов: outer и inner (где оно выполняется - на картинке)


Давайте напишем Middleware, который будет специально замедлять работу бота на n секунд

In [ ]:
import asyncio
from typing import Any, Callable, Dict, Awaitable
from aiogram import BaseMiddleware
from aiogram.types import TelegramObject

class SlowpokeMiddleware(BaseMiddleware):
    def __init__(self, sleep_sec: int):
        self.sleep_sec = sleep_sec

    async def __call__(
            self,
            handler: Callable[[TelegramObject, Dict[str, Any]], Awaitable[Any]],
            event: TelegramObject,
            data: Dict[str, Any],
    ) -> Any:
        await asyncio.sleep(self.sleep_sec)
        result = await handler(event, data)
        print(f"Handler was delayed by {self.sleep_sec} seconds")
        return result

In [ ]:
group_router = Router()
group_router.message.filter(
    ChatTypeFilter(chat_type=["group", "supergroup"])
)
group_router.message.middleware(SlowpokeMiddleware(sleep_sec=5))

Вопрос: какой тут middleware: inner или outer?

Ответ: inner. Чтобы сделать его outer, то мы делаем outer_middleware

О чем поговорим на семинаре?

* Конечные автоматы - что это и как их использовать в телеге

* Инлайн-режим - как к картинкам, сообщениям и так далее добавить кнопки!

И все это будет на примере оформления заказа в одном ресторане)

## Попугай дня

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/Ara_militaris_-London_Zoo-8a.jpg/1280px-Ara_militaris_-London_Zoo-8a.jpg)

Это солдатский ара. Назван так, потому что его внешний вид напоминает солдатскую униформу (интересно где)

К сожалению, их численность сейчас насчитывает около 5000 особей из-за уничтожения их естественной среды обитания. Поэтому считается одним из самых дорогих попугаев для заведения, но тем не менее, спокойно размножаются в неволе

Они одни из самых долгоживущий ар - живут до 60 лет, но чтобы их содержать нужно очень много с ними тратить времени. А также у них может быть проблема бутылочного горлышка в будущем (недостаточное разнообразие генов и их изменений приводит к вырождению рода)